In [ ]:
import pickle
import sys
import torch
import torch_geometric

with open('kirc_random_nodes_ui_pytorch.pkl', "rb") as f:
    data = pickle.load(f)   

print(data[0].node_feature_labels)
# First transformations & min/max finding
x_max_expr_all = sys.float_info.min
x_max_methyl_all = sys.float_info.min
x_min_expr_all = sys.float_info.max
x_min_methyl_all = sys.float_info.max

# First transformations & min/max finding
for graph in data:
    graph.y = graph.y.to(torch.float32)
    graph.edge_index = torch_geometric.utils.to_undirected(graph.edge_index)
    graph.edge_index = torch_geometric.utils.sort_edge_index(graph.edge_index)
    graph.edge_index = graph.edge_index.to(torch.long)
    del graph.node_labels
    del graph.node_ids
    del graph.node_feature_labels
    del graph.edge_ids
    del graph.edge_attr_labels
    del graph.graph_id

    x_max_expr, x_max_methyl = torch.amax(graph.x, dim=0).tolist()
    x_min_expr, x_min_methyl = torch.amin(graph.x, dim=0).tolist()

    if x_max_expr_all < x_max_expr:
        x_max_expr_all = x_max_expr

    if x_min_expr_all > x_min_expr:
        x_min_expr_all = x_min_expr

    if x_max_methyl_all < x_max_methyl:
        x_max_methyl_all = x_max_methyl

    if x_min_methyl_all > x_min_methyl:
        x_min_methyl_all = x_min_methyl

max_tensor_all = torch.tensor([x_max_expr, x_max_methyl])
min_tensor_all = torch.tensor([x_min_expr, x_min_methyl])

# Normalize
for graph in data:
    graph.x = (graph.x - min_tensor_all) / (max_tensor_all - min_tensor_all)


print(data)

with open('../kirc_random_nodes_preprocessed_all.pkl', "wb") as f:
    pickle.dump(data, f)

In [ ]:
#Get expression data (mRNA) and store
data_expr = [data[i].clone() for i in range(len(data))]
for i in range(len(data_expr)):
    data_expr[i].x = data[i].x[:,0].unsqueeze(1) 

print(data_expr)

with open('../kirc_random_nodes_preprocessed_expr.pkl', "wb") as f:
    pickle.dump(data_expr, f)
           

NameError: name 'data' is not defined

In [ ]:
#Get Methylation data and store
data_methyl = [data[i].clone() for i in range(len(data))]
for i in range(len(data_expr)):
    data_methyl[i].x = data[i].x[:,1].unsqueeze(1) 

print(data_methyl)

with open('../kirc_random_nodes_preprocessed_methyl.pkl', "wb") as f:
    pickle.dump(data_methyl, f)
           